In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import bilby

import sys
sys.path.insert(0,'../src')
from util import *
from cross_correlation import *
from postprocessing import *
from pe import *
from constants import *

In [ ]:
matplotlib.rcParams.update({'font.size':15})

# Analysis parameters

In [ ]:
deltaF=0.25
segmentDuration=64

# Noise only analysis

## Read and check data

In [ ]:
# read data

dat=np.load('data/colored_noise_H1.npy')
t1,h1=dat[:,0],dat[:,1]
d1=TimeSeries(t1,h1)

dat=np.load('data/colored_noise_H2.npy')
t2,h2=dat[:,0],dat[:,1]
d2=TimeSeries(t2,h2)

jobDuration=d2.times[-1]-d2.times[0]+d2.deltaT

In [ ]:
# Sanity check PSDs
nperseg=int(d1.Fs/deltaF)
freqs,P1_test = welch_psd(d1.data,nperseg=nperseg,window='Hann',fs=d1.Fs)
freqs,P2_test = welch_psd(d2.data,nperseg=nperseg,window='Hann',fs=d2.Fs)

plt.loglog(freqs,P1_test,label='H1')
plt.loglog(freqs,P2_test,label='H2')
plt.xlim(20,1024)
plt.ylim(1e-47,1e-45)
plt.legend()

## Cross correlation analysis

In [ ]:
# run cross correlation analysis
Ys,sigs,Y_ft,var_ft,segmentStartTimes,freqs=cross_correlation(d1,
                                                       d2,
                                                       segmentDuration,
                                                       deltaF,
                                                       verbose=True,
                                                       doOverlap=True,
                                                       alpha=0,
                                                       fref=25,
                                                       orf_file=None,
                                                       fmin=20,
                                                       fmax=1000)


In [ ]:
# combine spectra over times
Y_f,var_f=postprocessing_spectra(Y_ft,var_ft,jobDuration,segmentDuration,
                                 deltaF,d1.deltaT)

In [ ]:
# plot CC spectra
plt.loglog(freqs[f_filter],np.abs(Y_f[f_filter]))
plt.loglog(freqs[f_filter],np.sqrt(var_f[f_filter]))

In [ ]:
# compute optimal SNR

y,s=calc_Y_sigma_from_Yf_varf(np.real(Y_f[f_filter]),
                          var_f[f_filter],
                          freqs=freqs[f_filter],
                          alpha=2./3,
                          fref=25)

print('Optimal SNR [alpha=2/3]: %f'%(y/s))

In [ ]:
# Plot SNR vs alpha

alphas=np.linspace(-5,5,100)

snrs=np.zeros(alphas.shape)
for ii,a in enumerate(alphas):
    y,s=calc_Y_sigma_from_Yf_varf(np.real(Y_f[f_filter]),
                          var_f[f_filter],
                          freqs=freqs[f_filter],
                          alpha=a,
                          fref=25)
    snrs[ii]=y/s
plt.plot(alphas,snrs)
plt.axvline(2./3)
plt.xlabel('alpha')
plt.ylabel('SNR')

## Parameter estimation with power law model

In [ ]:
# A few simple setup steps
label = 'GWB_powerlaw'
outdir = 'outdir'

cleanup_dir(outdir)

Amin,Amax,alpha_min,alpha_max=1e-8,1e-3,-5,5
    

fref=25

likelihood = BasicPowerLawGWBLikelihood(Y_f[f_filter],var_f[f_filter],freqs[f_filter],fref)
priors = dict(A=bilby.core.prior.LogUniform(Amin,Amax, 'A'),
              alpha=bilby.core.prior.Uniform(alpha_min,alpha_max, 'alpha'))

# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=500,
    walks=10, outdir=outdir, label=label,maxmcmc=10000)
result.plot_corner()

# Noise + alpha=2/3 analysis


## Read and check data

In [ ]:
dat=np.load('data/colored_noise_plus_signal_H1.npy')
t1,h1=dat[:,0],dat[:,1]
d1=TimeSeries(t1,h1)

dat=np.load('data/colored_noise_plus_signal_H2.npy')
t2,h2=dat[:,0],dat[:,1]
d2=TimeSeries(t2,h2)

jobDuration=d2.times[-1]-d2.times[0]+d2.deltaT

In [ ]:
# Sanity check PSDs
nperseg=int(d1.Fs/deltaF)
freqs,P1_test = welch_psd(d1.data,nperseg=nperseg,window='Hann',fs=d1.Fs)
freqs,P2_test = welch_psd(d2.data,nperseg=nperseg,window='Hann',fs=d2.Fs)

plt.loglog(freqs,P1_test,label='H1')
plt.loglog(freqs,P2_test,label='H2')
plt.xlim(20,1024)
plt.ylim(1e-47,1e-45)
plt.legend()

## Cross correlation analysis

In [ ]:
Ys,sigs,Y_ft,var_ft,segmentStartTimes,freqs=cross_correlation(d1,
                                                       d2,
                                                       segmentDuration,
                                                       deltaF,
                                                       verbose=False,
                                                       doOverlap=True,
                                                       alpha=0,
                                                       fref=25,
                                                       orf_file=None,
                                                      fmin=20,fmax=1000)

In [ ]:
# combine spectra over times

Y_f,var_f=postprocessing_spectra(Y_ft,var_ft,jobDuration,segmentDuration,
                                 deltaF,d1.deltaT)

In [ ]:
plt.loglog(freqs,np.abs(Y_f),label='Y',alpha=0.3)
plt.loglog(freqs,np.sqrt(var_f),label=r'$\sigma$',color='blue')
plt.loglog(freqs,1e-7*(freqs/25)**(2./3),linestyle='--',label='Injection',color='red')
plt.legend()
plt.xlabel('Frequency [Hz]')
plt.ylabel(r'$\Omega_{\rm GW}$')

In [ ]:
# compute optimal SNR

y,s=calc_Y_sigma_from_Yf_varf(np.real(Y_f[f_filter]),
                          var_f[f_filter],
                          freqs=freqs[f_filter],
                          alpha=2./3,
                          fref=25)

print('Optimal SNR [alpha=2/3]: %f'%(y/s))


In [ ]:
alphas=np.linspace(-5,5,100)

snrs=np.zeros(alphas.shape)
for ii,a in enumerate(alphas):
    y,s=calc_Y_sigma_from_Yf_varf(np.real(Y_f[f_filter]),
                          var_f[f_filter],
                          freqs=freqs[f_filter],
                          alpha=a,
                          fref=25)
    snrs[ii]=y/s
plt.plot(alphas,snrs)
plt.axvline(2./3)
plt.xlabel('alpha')
plt.ylabel('SNR')

## PE with power law model

In [ ]:
# A few simple setup steps
label = 'GWB_powerlaw'
outdir = 'outdir'

cleanup_dir(outdir)

Amin,Amax,alpha_min,alpha_max=1e-8,1e-3,-5,5
    

fref=25

likelihood = BasicPowerLawGWBLikelihood(Y_f,var_f,freqs,fref)
priors = dict(A=bilby.core.prior.LogUniform(Amin,Amax, 'A'),
              alpha=bilby.core.prior.Uniform(alpha_min,alpha_max, 'alpha'))

injection_parameters=dict(A=1e-7,alpha=2./3)

# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=500,
    walks=10, outdir=outdir, label=label,maxmcmc=10000,injection_parameters=injection_parameters)
result.plot_corner()

## PE with broken power law model

In [ ]:
# A few simple setup steps
label = 'GWB_brokenpowerlaw'
outdir = 'outdir'

cleanup_dir(outdir)

Amin,Amax = 1e-8,1e-3
alpha1_min,alpha1_max=-5,5
alpha2_min,alpha2_max=-5,5
fbreak_min,fbreak_max=30,100
    

fref=25

baselines=[Baseline('test',Y_f,var_f,freqs)]

likelihood = BrokenPowerLawGWBLikelihood(baselines,fref)
priors = dict(A=bilby.core.prior.Uniform(Amin,Amax, 'A'),
              alpha1=bilby.core.prior.Uniform(alpha1_min,alpha1_max, 'alpha1'),
              alpha2=bilby.core.prior.Uniform(alpha2_min,alpha2_max, 'alpha2'),
              fbreak=bilby.core.prior.Uniform(fbreak_min,fbreak_max, 'fbreak'))

injection_parameters=dict(A=1e-7,alpha1=2./3,alpha2=2./3,fbreak=None)


# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=500,
    walks=10, outdir=outdir, label=label,maxmcmc=10000,injection_parameters=injection_parameters)
result.plot_corner()